# Setup

In [1]:
import random

import pandas as pd
import spotipy
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth
from tqdm.notebook import tqdm

In [ ]:
load_dotenv()

In [31]:
scope = "user-library-read,playlist-modify-public"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [4]:
playlists = sp.current_user_playlists()["items"]

In [ ]:
playlist_uris = {playlist["name"]: playlist["uri"] for playlist in playlists}
playlist_uris

In [6]:
driving = playlist_uris["Driving"]
test = playlist_uris["test"]

# Alphabetize

In [23]:
number_of_songs = sp.playlist_items(driving, limit=1)["total"]

In [ ]:
offsets = range(0, number_of_songs, 100)

In [ ]:
all_tracks = []
for offset in tqdm(offsets):
    all_tracks.extend(
        sp.playlist_items(
            driving, fields=["items(track(name,artists(name)))"], offset=offset
        )["items"]
    )

In [ ]:
len(all_tracks)

In [25]:
clean_all_tracks = []
for track in all_tracks:
    track = track["track"]
    name = track["name"]
    artists = ", ".join([artist["name"] for artist in track["artists"]])
    clean_all_tracks.append((name, artists))

In [26]:
songs_df = (
    pd.DataFrame(clean_all_tracks, columns=["name", "artists"])
    .reset_index()
    .sort_values("name", ascending=True)
)

In [19]:
ordered_song_order = list(songs_df["index"])

In [ ]:
with tqdm(total=len(ordered_song_order)) as t:
    while len(ordered_song_order) > 0:
        song_to_move = ordered_song_order.pop()

        for i, value in enumerate(ordered_song_order):
            if value < song_to_move:
                ordered_song_order[i] += 1

        sp.playlist_reorder_items(driving, song_to_move, 0)
        t.update()

In [ ]:
songs_df[songs_df.duplicated(["name"])]

# Now shuffle them all up

In [ ]:
number_of_songs = sp.playlist_items(driving, limit=1)["total"]
number_of_songs

In [33]:
playlist_song_order = list(range(number_of_songs))

In [ ]:
random.shuffle(playlist_song_order)
playlist_song_order

In [ ]:
with tqdm(total=number_of_songs) as t:
    while len(playlist_song_order) > 0:
        song_to_move = playlist_song_order.pop()

        for i, value in enumerate(playlist_song_order):
            if value < song_to_move:
                playlist_song_order[i] += 1

        sp.playlist_reorder_items(driving, song_to_move, 0)
        t.update()